In [33]:
import numpy as np
import pandas as pd
import tableauserverclient as TSC
from pandleau import *

In [9]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [11]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,981,989,998,1004,1029,1050,1076,1099,1122,1137
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,762,762,762,763,763,763,763,764,764,764
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,60,61,69,70,70,71,74,81,84,86


In [12]:
dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [13]:
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
34841,NaN,Sao Tome and Principe,0.186360,6.613081,5/31/20,483
34842,NaN,Yemen,15.552727,48.516388,5/31/20,323
34843,NaN,Comoros,-11.645500,43.333300,5/31/20,106
34844,NaN,Tajikistan,38.861034,71.276093,5/31/20,3930


In [14]:
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [15]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [16]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
34841,NaN,Sao Tome and Principe,0.186360,6.613081,5/31/20,483,12,68.0
34842,NaN,Yemen,15.552727,48.516388,5/31/20,323,80,NaN
34843,NaN,Comoros,-11.645500,43.333300,5/31/20,106,2,26.0
34844,NaN,Tajikistan,38.861034,71.276093,5/31/20,3930,47,NaN


In [17]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [18]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
34841,NaN,Sao Tome and Principe,0.186360,6.613081,2020-05-31,483,12,68.0
34842,NaN,Yemen,15.552727,48.516388,2020-05-31,323,80,NaN
34843,NaN,Comoros,-11.645500,43.333300,2020-05-31,106,2,26.0
34844,NaN,Tajikistan,38.861034,71.276093,2020-05-31,3930,47,NaN


In [19]:
full_table.isna().sum()

Province/State    24235
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          3668
dtype: int64

In [20]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

In [21]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

In [22]:
full_table = full_table[~(ship_rows)]

In [23]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

C:\Users\Rakesh Raj\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
34841,NaN,Sao Tome and Principe,0.186360,6.613081,2020-05-31,483,12,68.0,403.0
34842,NaN,Yemen,15.552727,48.516388,2020-05-31,323,80,0.0,243.0
34843,NaN,Comoros,-11.645500,43.333300,2020-05-31,106,2,26.0,78.0
34844,NaN,Tajikistan,38.861034,71.276093,2020-05-31,3930,47,0.0,3883.0


In [25]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

C:\Users\Rakesh Raj\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [26]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
24361,2020-05-31,West Bank and Gaza,448,3,372.0,73.0
24362,2020-05-31,Western Sahara,9,1,6.0,2.0
24363,2020-05-31,Yemen,323,80,0.0,243.0
24364,2020-05-31,Zambia,1057,7,779.0,271.0


In [38]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
temp.to_csv("temp.csv")

C:\Users\Rakesh Raj\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [30]:
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

In [31]:
mask

0         True
1        False
2        False
3        False
4        False
         ...  
24361    False
24362    False
24363    False
24364    False
24365    False
Name: Country/Region, Length: 24366, dtype: bool

In [45]:
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

In [37]:
temp.loc[mask, 'Confirmed']

0       NaN
131     NaN
262     NaN
393     NaN
524     NaN
         ..
23711   NaN
23842   NaN
23973   NaN
24104   NaN
24235   NaN
Name: Confirmed, Length: 186, dtype: float64

In [36]:
temp

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
24361,Zimbabwe,2020-05-27,76.0,0.0,0.0
24362,Zimbabwe,2020-05-28,17.0,0.0,3.0
24363,Zimbabwe,2020-05-29,0.0,0.0,0.0
24364,Zimbabwe,2020-05-30,25.0,0.0,1.0


In [39]:
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

In [40]:
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

In [41]:
# filling na with 0
full_grouped = full_grouped.fillna(0)

In [42]:
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

In [46]:
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)
full_grouped['New deaths'] = full_grouped['New deaths'].apply(lambda x: 0 if x<0 else x)
full_grouped['New recovered'] = full_grouped['New recovered'].apply(lambda x: 0 if x<0 else x)

In [47]:
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
24361,2020-05-31,West Bank and Gaza,448,3,372.0,73.0,1,0,4
24362,2020-05-31,Western Sahara,9,1,6.0,2.0,0,0,0
24363,2020-05-31,Yemen,323,80,0.0,243.0,13,3,0
24364,2020-05-31,Zambia,1057,7,779.0,271.0,0,0,0


In [48]:
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv')

In [50]:
df = pandleau(full_grouped)
df.to_tableau('mydata.hyper', add_index=False)

processing table: 24366it [00:01, 14284.37it/s]
